In [1]:
from pathlib import Path
import pandas as pd
import re
import yaml
import shutil

## Recursively navigate the target directory to location all images

In [2]:
with open(Path("./config/config.yaml"), "r") as f:
    config = yaml.safe_load(f)

input_dir = Path(config["input_dir"]).absolute()
image_keyword = config["image_keyword"]
segmentation_keyword = config["segmentation_keyword"]
image_format = config["image_format"]

In [3]:
path_atlas = Path(config["path_atlas"]).absolute()
wsl_mount = config["wsl_mount"]
path_output = Path(config["path_output"]).absolute()

In [4]:
# --- IGNORE ---

# OPTIONAL CLEANUP

shutil.rmtree(path_output, ignore_errors=True)

In [5]:
if input_dir.exists():
    print(f"Target directory found: {input_dir}")
else:
    raise FileNotFoundError(f"Target directory not found: {input_dir}")

Target directory found: c:\Users\m\Documents\run-synthmorph\input


In [6]:
def is_integer_folder(folder_name):
    """
    Check whether the folder name is an integer.

    Parameters
    ----------
    folder_name : str
        Name of the folder to check.

    Returns
    -------
    bool
        True if the folder name is an integer, False otherwise.
    """
    return re.fullmatch(r"\d+", folder_name) is not None


def find_extractions(base_dir):
    """
    Find all extraction folders in the base directory.

    Parameters
    ----------
    base_dir : Path
        The base directory to search for extraction folders.

    Returns
    -------
    list[Path]
        A list of paths to the extraction folders.
    """
    extractions = []
    for extraction_folder in base_dir.iterdir():
        if extraction_folder.is_dir():
            extractions.append(extraction_folder)
    return extractions


def find_subjects(extraction_folder):
    """
    Find all subject folders in the extraction folder.

    Parameters
    ----------
    extraction_folder : Path
        The extraction folder to search for subject folders.

    Returns
    -------
    list[Path]
        A list of paths to the subject folders.
    """
    subjects = []
    for subject_folder in extraction_folder.iterdir():
        if subject_folder.is_dir() and is_integer_folder(subject_folder.name):
            subjects.append(subject_folder)
    return subjects


def find_image_and_segmentation(subject_folder):
    """
    Find the image and segmentation files in subject/*/input.

    Parameters
    ----------
    subject_folder : Path
        The subject folder to search for NIfTI files.

    Returns
    -------
    tuple(Path, Path) | None
        The paths to the image and segmentation files if found, None otherwise.
    """
    for path in subject_folder.rglob("input"):
        if path.is_dir():
            image_file = None
            seg_file = None
            for file in path.glob(f"*{image_format}"):
                if image_keyword in file.name:
                    image_file = file
                elif segmentation_keyword in file.name:
                    seg_file = file
    return (image_file, seg_file) if image_file and seg_file else None

In [7]:
data = []
extractions = find_extractions(input_dir)
for extraction_folder in extractions:
    extraction_name = extraction_folder.name
    subjects = find_subjects(extraction_folder)
    for subject_folder in subjects:
        folder_id = subject_folder.name
        nii_path = find_image_and_segmentation(subject_folder)
        if nii_path:
            data.append(
                {
                    "extraction_path": extraction_folder,
                    "extraction": extraction_name,
                    "folder_id": folder_id,
                    "image_path": nii_path[0],
                    "image_name": nii_path[0].name,
                    "segmentation_path": nii_path[1],
                    "segmentation_name": nii_path[1].name,
                }
            )
df = pd.DataFrame(data)
df["subject_id_1"] = df["image_name"].str.split("_").str[0]
df["subject_id_2"] = df["image_name"].str.split("_").str[1]

In [8]:
df

,extraction_path,extraction,folder_id,image_path,image_name,segmentation_path,segmentation_name,subject_id_1,subject_id_2
0,c:\Users\m\Documents\run-synthmorph\input\capt...,captk-feature-extraction 1,26,c:\Users\m\Documents\run-synthmorph\input\capt...,C1031970_225_T1_ss_norm.nii.gz,c:\Users\m\Documents\run-synthmorph\input\capt...,C1031970_225_SegmWT.nii.gz,C1031970,225
1,c:\Users\m\Documents\run-synthmorph\input\capt...,captk-feature-extraction 2,27,c:\Users\m\Documents\run-synthmorph\input\capt...,C1072314_305_T1_ss_norm.nii.gz,c:\Users\m\Documents\run-synthmorph\input\capt...,C1072314_305_SegmWT.nii.gz,C1072314,305
2,c:\Users\m\Documents\run-synthmorph\input\capt...,captk-feature-extraction 3,26,c:\Users\m\Documents\run-synthmorph\input\capt...,C111930_194_T1_ss_norm.nii.gz,c:\Users\m\Documents\run-synthmorph\input\capt...,C111930_194_SegmWT.nii.gz,C111930,194
3,c:\Users\m\Documents\run-synthmorph\input\capt...,captk-feature-extraction 4,26,c:\Users\m\Documents\run-synthmorph\input\capt...,C114759_551_T1_ss_norm.nii.gz,c:\Users\m\Documents\run-synthmorph\input\capt...,C114759_551_SegmWT.nii.gz,C114759,551
4,c:\Users\m\Documents\run-synthmorph\input\capt...,captk-feature-extraction 5,26,c:\Users\m\Documents\run-synthmorph\input\capt...,C116850_624_T1_ss_norm.nii.gz,c:\Users\m\Documents\run-synthmorph\input\capt...,C116850_624_SegmWT.nii.gz,C116850,624


## Save shell script to output directory

In [9]:
path_trans_img = path_output / "transformed_img"
path_trans = path_output / "transformations"
path_trans_seg = path_output / "transformed_seg"

In [10]:
if wsl_mount:

    def convert_path(path):
        # convert Windows path to WSL path
        p = path.as_posix()
        if re.match(r"^[A-Za-z]:", p):
            drive = p[0].lower()
            p = p.replace(f"{p[0]}:", f"/mnt/{drive}")
            p = p.replace("\\", "/")
        return p
else:

    def convert_path(path):
        return str(path)

In [11]:
input_dir_env = convert_path(input_dir)
path_atlas_env = convert_path(path_atlas)
path_output_env = convert_path(path_output)
path_trans_img_env = convert_path(path_trans_img)
path_trans_env = convert_path(path_trans)
path_trans_seg_env = convert_path(path_trans_seg)

In [12]:
def get_registration_options(register_config):
    with open(Path(register_config), "r") as file:
        register_params = yaml.safe_load(file)
    reg_method = register_params.get("m_model")
    reg_options = ["mri_synthmorph register"]
    for k, v in register_params.items():
        if v is not None:
            flag = k.split("_")[0]
            reg_options.append(f"-{flag} {v}")
    return reg_method, tuple(reg_options)


def get_apply_options(apply_config):
    with open(Path(apply_config), "r") as file:
        apply_params = yaml.safe_load(file)
    apply_options = ["mri_synthmorph apply"]
    for k, v in apply_params.items():
        if v is not None:
            flag = k.split("_")[0]
            apply_options.append(f"-{flag} {v}")
    return tuple(apply_options)

In [13]:
reg_method, reg_cmd = get_registration_options("./config/register.yaml")
apply_cmd = get_apply_options("./config/apply.yaml")

In [14]:
# initialize bash script and output directory
script_lines = [
    "#!/bin/bash",
    "set -e",
    f"export INPUT='{input_dir_env}'",
    f"export ATLAS='{path_atlas_env}'",
    f"export OUTPUT='{path_output_env}'",
    f"export TRANS_IMG='{path_trans_img_env}'",
    f"export TRANS='{path_trans_env}'",
    f"export TRANS_SEG='{path_trans_seg_env}'",
]
for d in [path_trans_img, path_trans, path_trans_seg]:
    Path(d).mkdir(parents=True, exist_ok=True)

In [15]:
# for each image
for idx, row in df.iterrows():
    path_image = convert_path(row["image_path"])
    path_seg = convert_path(row["segmentation_path"])
    subject_id_1 = row["subject_id_1"]
    subject_id_2 = row["subject_id_2"]

    lta_name = f"{subject_id_1}_{subject_id_2}_{reg_method}.lta"
    lta_path = "$TRANS/" + lta_name

    # registration command
    reg_line = list(reg_cmd)
    reg_line.append(f"-t {lta_path}")
    reg_line.append(f"-o $TRANS_IMG/{row['image_name']} -v '{path_image}' $ATLAS")

    # apply command
    apply_line = list(apply_cmd)
    apply_line.append(f"{lta_path} '{path_seg}' $TRANS_SEG/{row['segmentation_name']}")

    # append to script lines
    script_lines.append(" ".join(str(x) for x in reg_line))
    script_lines.append(" ".join(str(x) for x in apply_line))

In [16]:
script_path = path_output / "run_synthmorph.sh"
with open(script_path, "w", newline="\n") as f:
    f.write("\n".join(script_lines))
print(f"Shell script written to {script_path}")

Shell script written to c:\Users\m\Documents\run-synthmorph\output\run_synthmorph.sh
